In [1]:
import numpy as np
import tensorflow as tf

In [2]:
raw_Data = np.loadtxt('Audiobooks_data.csv', delimiter=',')

unscaled_inputs = raw_Data[:, 1:-1]
targets = raw_Data[:, -1]

In [3]:
num_of_ones = int(np.sum(targets))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets.shape[0]):
    if targets[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_of_ones:
            indices_to_remove.append(i)

unscaled_inputs_equal_prior = np.delete(unscaled_inputs, indices_to_remove, axis=0)
targets_equal_prior = np.delete(targets, indices_to_remove, axis=0)

In [4]:
from sklearn.preprocessing import scale
scaled_inputs = scale(X=unscaled_inputs_equal_prior)

In [5]:
shuffled_indicies = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indicies)

shuffled_inputs = scaled_inputs[shuffled_indicies]
shuffled_targets = targets_equal_prior[shuffled_indicies]

In [6]:
samples_count = shuffled_indicies.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count : train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count : train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count :]
test_targets = shuffled_targets[train_samples_count + validation_samples_count :]

print(np.sum(train_targets), train_samples_count, (np.sum(train_targets) / train_samples_count))
print(np.sum(validation_targets), validation_samples_count, (np.sum(validation_targets) / validation_samples_count))
print(np.sum(test_targets), test_samples_count, (np.sum(test_targets) / test_samples_count))

1776.0 3579 0.49622799664710815
242.0 447 0.5413870246085011
219.0 448 0.4888392857142857


In [7]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [8]:
npz_train = np.load('Audiobooks_data_train.npz')
npz_validation = np.load('Audiobooks_data_validation.npz')
npz_test = np.load('Audiobooks_data_test.npz')

In [9]:
train_inputs = npz_train['inputs'].astype(np.float64)
train_targets = npz_train['targets'].astype(np.int32)

validation_inputs = npz_validation['inputs'].astype(np.float64)
validation_targets = npz_validation['targets'].astype(np.int32)

test_inputs = npz_test['inputs'].astype(np.float64)
test_targets = npz_test['targets'].astype(np.int32)

In [10]:
input_size = 10
output_size = 2
hidden_layer_size = 10

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='ADAM', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
early_stopping = tf.keras.callbacks.EarlyStopping()

model.fit(train_inputs, 
          train_targets, 
          batch_size=100, 
          epochs=100, 
          callbacks=[early_stopping], 
          validation_data=(validation_inputs, validation_targets), 
          verbose=2)

Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow 

In [ ]:
model.evaluate(test_inputs, test_targets)